# Homework A (Bonus - 1 points)
We want to study the impact of discount factor using the endowments (income) in the first and second periods, and interest rate given below on the first and second period consumption, and saving amount. 

y_0 = 1, 
y_1 = 1, 
r = 0.05

Use values from 0.9 to 0.99 with 0.01 increments for beta and create table to show the values of c_0, c_1, a, and utility for each value of beta. Explain how consumption and savings change with the increase in beta. Why?

To solve, you need to insert equations for optimal level of c_0, c_1, a and use the logarithmich utility function to derive the values of c_0, c_1, a, and utility at the optimal level of c_0 and c_1.

In [ ]:
# Make graphs appear within the notebook
%matplotlib inline

# Import numerical computations library
import numpy as np

# Import plotting library
import matplotlib.pyplot as plt

# Import minimization function
from scipy.optimize import minimize

# Import optimization function
import scipy.optimize as opt

# Parameters

params = {'y_0':1, 'y_1':1, 'r':0.05, 'beta':0.9}


def SolveUtilityMaximizationProblem(y_0=1, y_1=1, β=0.95, r=0.05, figsize=(5, 5)):

    c_0_max = y_0 + y_1/(1+r)
    c_0 = np.linspace(0, c_0_max)
    
    c_0_opt = 1/(1+β) * (y_0 + y_1/(1+r))
    c_1_opt = β*(1+r)*c_0_opt
    U_opt = np.log(c_0_opt) + β*np.log(c_1_opt)
    
    fig, ax = plt.subplots(figsize=figsize)

    PlotBudgetSet(y_0, y_1, r, fig_ax=(fig, ax))
    PlotIndifferenceCurves(y_0, y_1, β, levels=[U_opt], fig_ax=(fig, ax))

    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    
    p = ax.plot(c_0, β*(1+r)*c_0, linewidth=2, label='Euler equation')
    color = p[0].get_color()
    
    ax.plot(c_0_opt, c_1_opt, 'o', color=color)

    plt.vlines(c_0_opt, 0, c_1_opt, color=color, linestyle='dashed', linewidth=1)
    plt.hlines(c_1_opt, 0, c_0_opt, color=color, linestyle='dashed', linewidth=1)
    
    l = plt.legend(loc='upper right', fontsize=12)
    l.get_frame().set_linewidth(0)
    l.get_frame().set_alpha(1)
    
    if c_0_opt == y_0:
        plt.xticks((0, y_0, xmax), ('0', '$c_{t} = y_{t}$', '$c_{t}$'), fontsize=12)
        plt.yticks((0, y_1, ymax), ('', '$c_{t+1} = y_{t+1}$', '$c_{t+1}$'), fontsize=12)
    elif abs(c_0_opt-y_0) < xmax/20:
        if c_0_opt > y_0:
            plt.xticks((0, (y_0+c_0_opt)/2, xmax), ('0', '$c_{t} > y_{t}$', '$c_{t}$'), fontsize=12)
            plt.yticks((0, (y_1+c_1_opt)/2, ymax), ('', '$c_{t+1} < y_{t+1}$', '$c_{t+1}$'), fontsize=12)
        else:
            plt.xticks((0, (y_0+c_0_opt)/2, xmax), ('0', '$c_{t} < y_{t}$', '$c_{t}$'), fontsize=12)
            plt.yticks((0, (y_1+c_1_opt)/2, ymax), ('', '$c_{t+1} > y_{t+1}$', '$c_{t+1}$'), fontsize=12)
    else:
        plt.xticks((0, y_0, c_0_opt, xmax), ('0', '$y_{t}$', '$c_{t}$', '$c_{t}$'), fontsize=12)
        plt.yticks((0, y_1, c_1_opt, ymax), ('', '$y_{t+1}$', '$c_{t+1}$', '$c_{t+1}$'), fontsize=12)

    # plt.savefig('test.pdf')
    
    print("Consumer's income:")
    print('  y_0: {:.4f}'.format(y_0))
    print('  y_1: {:.4f}'.format(y_1))
    print('')
    
    print('Optimal consumption:')
    print('  c_0: {:.4f}'.format(c_0_opt))
    print('  c_1: {:.4f}'.format(c_1_opt))
    print('')
    
    if c_0_opt > y_0:
        print('c_0 > y_0: Consumer is a borrower')
    elif c_0_opt < y_0:
        print('c_0 < y_0: Consumer is a saver')
    else:
        print('c_0 = y_0: Consumer is neither a borrower nor a saver')
    print('')
        
    return None


TypeError: 'float' object cannot be interpreted as an integer